In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [1]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeWorkingHRDfParquet

import pandas as pd

fitbitHRWorkingDataPath = workingDataPath + 'fitbit/hr/'

# Get the list of all files and directories
exportFilePath = exportsDataPath + "fitbit/17-9-24/takeout-20240917T195619Z-001/Takeout/Fitbit/Global Export Data/"
dir_list = os.listdir(exportFilePath)


/home/chowder/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# read in the existing files and make the existing df
workingDataFiles = os.listdir(fitbitHRWorkingDataPath)
columnNames = ["sampleDT", "confidence", "value"]
dfSoFar = pd.DataFrame(columns=columnNames).set_index("sampleDT")

In [3]:
for dataFileName in workingDataFiles:
    dfSoFar = pd.concat([dfSoFar, pd.read_parquet(fitbitHRWorkingDataPath + dataFileName)]) 

dfSoFar = dfSoFar[~dfSoFar.index.duplicated(keep="first")].sort_index()

print(f"read in {len(dfSoFar)} rows from {len(workingDataFiles)} files")

read in 0 rows from 0 files


In [4]:
# get a list of unique dates in the index
#   removing the latest 3 days since they might be incomplete
if len(dfSoFar) > 0:
    datesSoFar = sorted(list(set(dfSoFar.index.date)))[:-3]
    print(datesSoFar[-1])
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]
    hrFilenames = [x for x in hrFilenames if pd.to_datetime(x[11:-5]).date() not in datesSoFar]
else:
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]

In [5]:
#takes like 50 seconds to make 16 million rows
import json
import pytz
from datetime import datetime

def to_iso(s):
    return '20' + s[6:8] + '-' + s[0:2] + '-' + s[3:5] + s[8:]

samplesCount = 0
samplesList = []
for fn in hrFilenames:
    data = json.load(open(exportFilePath + fn))
    for row in data:
        sampleDT = datetime.fromisoformat(to_iso(row["dateTime"]))
        samplesList.append([sampleDT, 
                            row["value"]["confidence"], 
                            row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 1_000_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 1000000 samples so far
added 2000000 samples so far
added 3000000 samples so far
added 4000000 samples so far
added 5000000 samples so far
added 6000000 samples so far
added 7000000 samples so far
added 8000000 samples so far
added 9000000 samples so far
added 10000000 samples so far
added 11000000 samples so far
added 12000000 samples so far
added 13000000 samples so far
added 14000000 samples so far
added 15000000 samples so far
added 16000000 samples so far
added 16052785 samples


In [6]:
import pandas as pd

columnNames = ["sampleDT", "confidence", "value"]
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf.index = fitbitHRdf.index.tz_localize('UTC')

fitbitHRdf.index = fitbitHRdf.index.tz_convert("US/Arizona")
fitbitHRdf.index = pd.to_datetime(fitbitHRdf.index)

fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

confidence    uint8
value         uint8
dtype: object

In [7]:
# concatenate the df's
# remove duplicate indexes
fitbitHRdf = pd.concat([dfSoFar, fitbitHRdf])
fitbitHRdf = fitbitHRdf[~fitbitHRdf.index.duplicated(keep="first")].sort_index()

In [8]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf)

16052785 is too many rows writing 0 to 365436
saved to a file named 2020-05-13T100219-0700_2020-06-15T175149-0700_9b3f3630_.parquet.gzip
15687349 is too many rows writing 365436 to 730872
saved to a file named 2020-06-15T175149-0700_2020-07-19T103931-0700_15ed86cc_.parquet.gzip
15321913 is too many rows writing 730872 to 1096308
saved to a file named 2020-07-19T103931-0700_2020-08-22T011641-0700_0c4966ee_.parquet.gzip
14956477 is too many rows writing 1096308 to 1461744
saved to a file named 2020-08-22T011641-0700_2020-09-25T101037-0700_efba1909_.parquet.gzip
14591041 is too many rows writing 1461744 to 1827180
saved to a file named 2020-09-25T101037-0700_2020-10-28T182707-0700_5f1b0c2b_.parquet.gzip
14225605 is too many rows writing 1827180 to 2192616
saved to a file named 2020-10-28T182707-0700_2020-12-01T022613-0700_ad3308cd_.parquet.gzip
13860169 is too many rows writing 2192616 to 2558052
saved to a file named 2020-12-01T022613-0700_2021-01-04T062220-0700_1bb757ba_.parquet.gzip
13